In [8]:
import bz2
import json
from glob import glob
from tqdm import tqdm
from elasticsearch import Elasticsearch

In [9]:
path = 'data/dump/elastic/nlp/crawler/*.json.bz2'

f_list = glob(path)

In [10]:
filename = 'data/dump/elastic/nlp/crawler/crawler-naver-economy-2010.json.bz2'
with bz2.open(filename) as fp:
    for l in fp.readlines():
        doc = json.loads(l.decode('utf-8'))
        print(doc)
        break

{'content': '교보증권이 오는 5일까지 제481회 ELS 1종을 발행합니다. \n\n삼성SDI 보통주와 현대중공업 보통주를 기초자산으로 삼는 만기 2년형 원금비보장형 상품으로 만기평가일까지 한 번도 최초기준주가대비 50%이상 하락한 적이 없으면 36%의 수익을 지급합니다. \n\n또한 매 4개월마다 조기상환기회를 부여해 두 종목 모두 최초기준주가의 90%(4,8개월), 85%(12,16개월), 80%(20개월) 이상인 경우 연 18%의 수익을 얻을 수 있습니다.\n\n남진철 교보증권 장외파생상품팀장은 "기초자산을 우량주로 구성하고 원금손실이 발생하는 구간을 최초기준주가의 50%수준으로 낮춰 안정성을 강조했다"고 밝혔습니다.\n\n최소청약금액은 100만원이며 기타 자세한 사항은 대표전화로 문의하면 됩니다. \n\n빠르고, 쉽고, 싼 휴대폰 주가조회 숫자 \'969\'+NATE/magicⓝ/SHOW/ez-i>\n\nTV를 통한 관심종목 조회 및 주식매매 서비스(데이터방송)>\n\n박진규기자 jkyu2002@wowtv.co.kr', 'date': '2010-02-03T19:41:00+09:00', 'document_id': '004-0002001885', 'html_content': '<div id="articleBodyContents">\n 교보증권이 오는 5일까지 제481회 ELS 1종을 발행합니다.\n <br/>\n <br/>\n 삼성SDI 보통주와 현대중공업 보통주를 기초자산으로 삼는 만기 2년형 원금비보장형 상품으로 만기평가일까지 한 번도 최초기준주가대비 50%이상 하락한 적이 없으면 36%의 수익을 지급합니다.\n <br/>\n <br/>\n 또한 매 4개월마다 조기상환기회를 부여해 두 종목 모두 최초기준주가의 90%(4,8개월), 85%(12,16개월), 80%(20개월) 이상인 경우 연 18%의 수익을 얻을 수 있습니다.\n <br/>\n <br/>\n 남진철 교보증권 장외파생상품팀장은 "기초자산을 우량주로 구성하고 원금손실이 발생하는 구간을

In [7]:
elastic = Elasticsearch(hosts='https://corpus.ncsoft.com:9200',
                        timeout=120,
                        http_auth='elastic:nlplab')

In [11]:
from uuid import uuid4

index = 'sentence-pool-ko'
column = 'expExample2Text'

bulk_data = []
with bz2.open('data/crawler-dictionary-example-naver-ja.json.bz2', 'rb') as fp:
    for l in tqdm(fp.readlines()):
        doc = json.loads(l.decode('utf-8'))

        item = {
            'source': doc['sourceDictnameKO'],
            'text': doc[column],
        }

        bulk_data += [{
            'update': {
                '_index': index,
                '_id': str(uuid4()),
            }
        }, {
            'doc': item,
            'doc_as_upsert': True,
        }]
        
        if len(bulk_data) > 1000:
            resp = elastic.bulk(
                index=index,
                body=bulk_data,
                refresh=True,
                params={'request_timeout': 2 * 60},
            )
            
            bulk_data = []
            
resp = elastic.bulk(
    index=index,
    body=bulk_data,
    refresh=True,
    params={'request_timeout': 2 * 60},
)
bulk_data = []

100%|██████████| 106274/106274 [00:26<00:00, 4025.19it/s]
